# PROJET : COMPARAISON DE TOUTES LES METHODES

# AKATCHI DANIEL p2412436


In [5]:
# VERSION R PURE 

simuler_X_R <- function(n, lambda, mu, sigma) {
  resultats <- numeric(n)
  
  for (i in 1:n) {
    N <- rpois(1, lambda)
    
    if (N > 0) {
      C_i <- rlnorm(N, meanlog = mu, sdlog = sigma)
      resultats[i] <- sum(C_i)
    } else {
      resultats[i] <- 0
    }
  }
  
  return(resultats)
}

cat("Fonction R pure chargée\n")


Fonction R pure chargée


In [1]:
library(Rcpp)

Sys.unsetenv("PKG_CXXFLAGS")
Sys.unsetenv("PKG_LIBS")

cat("Configuration Rcpp sans OpenMP \n")


Configuration Rcpp sans OpenMP 


In [2]:
#  VERSION RCPP cppFunction 

cppFunction('
NumericVector rpcomp2_cpp(int n, double lambda, double mu, double sigma) {
    NumericVector resultats(n);

    for(int i = 0; i < n; i++) {
        int N = R::rpois(lambda);
        double somme = 0.0;

        for(int j = 0; j < N; j++) {
            somme += R::rlnorm(mu, sigma);
        }

        resultats[i] = somme;
    }

    return resultats;
}
')

cat("Version Rcpp cppFunction chargée\n")


Version Rcpp cppFunction chargée


In [4]:
#  COMPILATION OPENMP (macOS clang + libomp) 

Sys.setenv(
  PKG_CXXFLAGS = "-Xpreprocessor -fopenmp -I/opt/homebrew/opt/libomp/include",
  PKG_LIBS     = "-L/opt/homebrew/opt/libomp/lib -lomp"
)

cat("Compilation OpenMP avec clang + libomp\n")

sourceCpp("rpcomp2_openmp.cpp")



Compilation OpenMP avec clang + libomp


In [8]:
# Compilation du fichier sourceCpp
sourceCpp("rpcomp2_source.cpp")
cat("Fichier rpcomp2_source.cpp compilé et fonction disponible\n")


Fichier rpcomp2_source.cpp compilé et fonction disponible


Sous macOS, le plugin Rcpp openmp ajoute -fopenmp, ce qui est incompatible avec Apple clang. J’ai donc supprimé le plugin et utilisé libomp avec les flags -Xpreprocessor -fopenmp -lomp, qui est la solution recommandée

In [9]:
#  COMPARAISON DES PERFORMANCES 


n <- 100000       # nombre de simulations
lambda <- 5
mu <- 0
sigma <- 1

cat(" COMPARAISON DES MÉTHODES \n")

# 1. R pur
cat("1. R pur... ")
t1 <- system.time(r1 <- simuler_X_R(n, lambda, mu, sigma))[3]
cat(round(t1, 3), "s\n")

# 2. Rcpp cppFunction
cat("2. Rcpp cppFunction... ")
t2 <- system.time(r2 <- rpcomp2_cpp(n, lambda, mu, sigma))[3]
cat(round(t2, 3), "s (x", round(t1/t2, 1), ")\n")

# 3. Rcpp sourceCpp
cat("3. Rcpp sourceCpp... ")
t3 <- system.time(r3 <- rpcomp2_source(n, lambda, mu, sigma))[3]
cat(round(t3, 3), "s (x", round(t1/t3, 1), ")\n")

# 4. OpenMP basic
cat("4. OpenMP basic... ")
t4 <- system.time(r4 <- rpcomp2_omp_basic(n, lambda, mu, sigma))[3]
cat(round(t4, 3), "s (x", round(t1/t4, 1), ")\n")

# 5. OpenMP static
cat("5. OpenMP static... ")
t5 <- system.time(r5 <- rpcomp2_omp_static(n, lambda, mu, sigma, 100))[3]
cat(round(t5, 3), "s\n")

# 6. OpenMP dynamic
cat("6. OpenMP dynamic... ")
t6 <- system.time(r6 <- rpcomp2_omp_dynamic(n, lambda, mu, sigma, 100))[3]
cat(round(t6, 3), "s\n")

# 7. OpenMP auto
cat("7. OpenMP auto... ")
t7 <- system.time(r7 <- rpcomp2_omp_auto(n, lambda, mu, sigma))[3]
cat(round(t7, 3), "s\n")


 COMPARAISON DES MÉTHODES 
1. R pur... 0.104 s
2. Rcpp cppFunction... 0.015 s (x 6.9 )
3. Rcpp sourceCpp... 0.017 s (x 6.1 )
4. OpenMP basic... 0.008 s (x 13 )
5. OpenMP static... 0.005 s
6. OpenMP dynamic... 0.004 s
7. OpenMP auto... 0.004 s


RÉSUMÉ DES GAIN DE TEMPS: 

In [10]:
cat("\n RÉSUMÉ DES GAIN DE TEMPS \n")
cat("Rcpp cppFunction x", round(t1/t2,1), "\n")
cat("Rcpp sourceCpp x", round(t1/t3,1), "\n")
cat("OpenMP basic x", round(t1/t4,1), "\n")
cat("OpenMP static x", round(t1/t5,1), "\n")
cat("OpenMP dynamic x", round(t1/t6,1), "\n")
cat("OpenMP auto x", round(t1/t7,1), "\n")



 RÉSUMÉ DES GAIN DE TEMPS 
Rcpp cppFunction x 6.9 
Rcpp sourceCpp x 6.1 
OpenMP basic x 13 
OpenMP static x 20.8 
OpenMP dynamic x 26 
OpenMP auto x 26 


commentaire et remarques:

Ce projet m’a permis de comprendre la différence entre R pur et C++ via Rcpp en termes de performance.
La configuration OpenMP sur macOS est un peu délicate à cause de la limitation d’Apple clang, mais j’ai réussi à contourner le problème avec libomp et les flags -Xpreprocessor -fopenmp -lomp.
La comparaison des différentes stratégies OpenMP (static, dynamic, auto) montre que le choix de la schedule peut influencer les performances, surtout quand la charge est inégale entre les threads.
Ces résultats me confirment que pour des simulations massives (≥100 000 itérations), la combinaison Rcpp + OpenMP est la plus efficace.

-R pur est simple mais lent pour de grandes simulations.
-Rcpp accélère déjà considérablement les calculs.
-OpenMP permet un gain supplémentaire massif et montre l’intérêt de la parallélisation.

Les trois schedules static, dynamic et auto sont toutes efficaces, dynamic et auto étant légèrement meilleures pour cette simulation spécifique.